In [13]:
# pytorch mlp for regression
from numpy import vstack
import numpy as np
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Tanh
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
 

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-2].astype('float32')
        self.y = df.values[:, -2:].astype('float32')
        # print(len(self.X[0]))
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 2))
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs,layer1, layer2, act_func):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, layer1)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = act_func()
        # second hidden layer
        self.hidden2 = Linear(layer1, layer2)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = act_func()
        # third hidden layer and output
        self.hidden3 = Linear(layer2, 2)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    print(CSVDataset(path).__getitem__(0))
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model, learn):
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=learn)
    # enumerate epochs
    for epoch in range(1000):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals, erroDistx, erroDisty = list(), list(), list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 2))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)

    predit = np.array(predictions)
    actu = np.array(actuals)
    act = actu.tolist()
    dif = predit-actu.tolist()
    # print(dif[:][0])
    # print('-----------------------------------------------------------------------------------------')
    # print(act)
    j=0
    while j < len(dif):
        erroDistx.append(dif[j][0])
        erroDisty.append(dif[j][1])
        # with open('erros_coord.csv', 'a') as f:
        #         f.write(f'{dif[j][0]},{dif[j][1]},{act[j][0]},{act[j][1]}\n')
        j+=1
    # print(erroDistx)
    # print('------------------------------------------')
    # print(erroDisty)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    menorx = min(erroDistx)
    maiorx = max(erroDistx)
    menory = min(erroDisty)
    maiory = max(erroDisty)
    return mse, mae, menorx, menory, maiorx, maiory
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat
 
# prepare the data
path = 'DataSet2.csv'
train_dl, test_dl = prepare_data(path)
# print(len(train_dl.dataset), len(test_dl.dataset))
# layers1 = 100
# layers2 = 110
# actFunc = Sigmoid
# learning = 0.001

                
# model = MLP(15,layers1,layers2,actFunc)
# evaluate_model(train_dl, model)

[array([-59., -56., -57., -52., -51., -59., -62., -57., -60., -58., -63.,
       -66., -58., -62., -62.], dtype=float32), array([2., 2.], dtype=float32)]


In [14]:
print(len(train_dl))

57


In [5]:
# with open('results41.csv', 'a') as f:
#     f.write('layer1,layer2,actFunc,learning,mse,mae\n') 
layers1 = 100
layers2 = 110
actFunc = Sigmoid
learning = 0.001

                
model = MLP(15,layers1,layers2,actFunc)
# train the model
train_model(train_dl, model,learning)
# evaluate the model


In [16]:
mse, mae, menorx, menory, maiorx, maiory = evaluate_model(train_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
print('MAE: %.3f' % (mae))

print(f'errox {menorx}')
print(f'errox {maiorx}')
print(f'erroy {menory}')
print(f'erroy {maiory}')
# make a single prediction (expect class=1)
row = [-46,-48,-52,-46,-46,-59,-91,-63,-63,-60,-64,-66,-60,-59,-68]
yhat = predict(row, model)
print( yhat)

MSE: 0.503, RMSE: 0.709
MAE: 0.347
errox -4.020050287246704
errox 4.135917663574219
erroy -6.574370861053467
erroy 4.280078411102295
[[3.736328 1.594575]]


In [12]:
import csv
import numpy as np
with open('CoordRNAPredict.csv','r') as b:
    reader = csv.reader(b)
    lis = list(reader)

inter =[]
row = []
flat_list1 = []
for xs in lis:
    for x in xs:
        flat_list1.append(x)

dist = list(map(int, flat_list1))
i=0
j=0
while i < len(dist):
    
    while j < len(dist):
        inter.append(dist[j])
        j+=1
        if j%15==0:
            break
    if len(inter)==0:
        break
    row.append(inter)
    inter = []
    i+=1
# print(row)
# k=0
# while k<len(row):
    
# make a single prediction (expect class=1)
# row = [-56,-54,-60,-54,-58,-60,-55,-59,-59,-54]
for x in row:
    # print(x)
    yhat = predict(x, model)
    print(yhat.tolist())
    with open('RNACoordPredict.csv', 'a') as f:
                f.write(f'{yhat[0][0]},{yhat[0][1]}\n')
    # print('Predicted: %.3f' % yhat)

[[1.9548107385635376, 3.1584692001342773]]
[[1.969470739364624, 2.117898941040039]]
[[1.9803640842437744, 1.8647125959396362]]
[[1.960516333580017, 2.4184730052948]]
[[1.9800595045089722, 2.008023977279663]]
[[2.005959987640381, 1.8431984186172485]]
[[1.981519103050232, 1.8489774465560913]]
[[2.0854873657226562, 1.8442150354385376]]
[[1.9805546998977661, 1.869470238685608]]
[[1.9783923625946045, 1.971673846244812]]
[[2.1124765872955322, 1.8872147798538208]]
[[2.0164501667022705, 1.8471322059631348]]
[[2.0088770389556885, 1.846310019493103]]
[[1.981807827949524, 1.8473049402236938]]
[[5.967357635498047, 1.9810062646865845]]
[[2.029372215270996, 1.8515859842300415]]
[[1.9974284172058105, 1.8513966798782349]]
[[1.9746818542480469, 1.9464493989944458]]
[[2.2769687175750732, 3.9777262210845947]]
[[2.4772822856903076, 2.1875338554382324]]
[[1.988822102546692, 1.8510397672653198]]
[[2.006638526916504, 1.8464889526367188]]
[[1.9535086154937744, 2.770786762237549]]
[[2.370900869369507, 1.833964